In [9]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [15]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/SDA_experiments/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/SDA_experiments/modules', '/tmp/tmpkxj2dfrg']


In [41]:
from datasets import load_from_disk,concatenate_datasets
from datasets import Dataset
import pandas as pd
from datasets import Dataset, DatasetDict


adapter_name="mlm_inv_baby"

# Define paths to the CSV files
unlabeled_target_data_path = "../../datasets/sa/books_baby/target_unlabelled.csv"
dev_target_data_path = "../../datasets/sa/books_baby/dev_target.csv"
test_target_data_path = "../../datasets/sa/books_baby/test_target.csv"

# Read the CSV files into DataFrames
df_unlabeled_target = pd.read_csv(unlabeled_target_data_path)
df_dev_target = pd.read_csv(dev_target_data_path)
df_test_target = pd.read_csv(test_target_data_path)

# Convert the DataFrames to dictionaries
data_dict_unlabeled_target = df_unlabeled_target.to_dict(orient="records")
data_dict_dev_target = df_dev_target.to_dict(orient="records")
data_dict_test_target = df_test_target.to_dict(orient="records")

# Create Dataset objects from the dictionaries
dataset_unlabeled_target = Dataset.from_dict({
    "sentence": [record["sentence"] for record in data_dict_unlabeled_target],
    
})

dataset_dev_target = Dataset.from_dict({
    "sentence": [record["sentence"] for record in data_dict_dev_target],
})

dataset_test_target = Dataset.from_dict({
    "sentence": [record["sentence"] for record in data_dict_test_target],
    "label": [record["label"] for record in data_dict_test_target]
})

# Combine the unlabeled_target and dev_target datasets into a DatasetDict for training
dataset_train = DatasetDict({
    "train": dataset_unlabeled_target,
    "dev": dataset_dev_target
})

# Print the datasets to verify
print("Training Dataset with Unlabeled and Dev Splits:")
print(dataset_train)

print("\nTest Target Dataset:")
print(dataset_test_target)



Training Dataset with Unlabeled and Dev Splits:
DatasetDict({
    train: Dataset({
        features: ['sentence'],
        num_rows: 1350
    })
    dev: Dataset({
        features: ['sentence'],
        num_rows: 150
    })
})

Test Target Dataset:
Dataset({
    features: ['sentence', 'label'],
    num_rows: 400
})


In [36]:
reload(config)


train_target = dataset_train
test_target = dataset_test_target
train_target['train'][1]
 

{'sentence': '"i was very disppointed in this product . i found it great that the containers would easily stack in the freezer , but this was the only good quality . i had to throw out all of my containers because they leaked and the plastic was obviously leaching . you could smell the plastic in the milk . i \'m not going to expose my baby to extra chemicals , especially when her liver is n\'t developed enough to fully detoxify them . i really wanted to love this product , but leaching plastic that leaks is not an acceptable option"'}

In [42]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [43]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig


adapter_config = SeqBnInvConfig(reduction_factor=2)

model.add_adapter(adapter_name, config=adapter_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_inv_baby             bottleneck        7,387,776       6.785       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 7979904 || all params: 116902074 || trainable%: 6.82614407679371


In [44]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Map:   0%|          | 0/1350 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (643 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 1350
    })
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 150
    })
})

In [46]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset['train'][444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [47]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Map:   0%|          | 0/1350 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1405
    })
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 161
    })
})

In [48]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets['train'][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] " my son, who was used to using a wood stool ( [MASK]'' tall with slick wood [MASK] ) fell climbing up [MASK] this because the top ( foot relocate ) [MASK] too small and it tipped on [MASK]! it's only 6'[MASK] wide on top spurs [MASK] [MASK] room khan clumsy [MASK] [MASK] to turn around on. [MASK] you push [MASK] in toward the toilet there is [MASK] left for the [MASK] to stand on. if you leave it out, [MASK] it takes up more room and the arc design is a waste. [MASK] down he also started to fall because it was such a narrow top. [MASK] feet [MASK] dangled as he'

'>>> sat on the toilet - not [MASK] near the top of this 8 1 [MASK] 2'' tall stool. medieval is [MASK] as being 10 raging'tall ( actually 8 1 / 2'[MASK] ) 14'' long ( correct, but top surface is only [MASK] 1 [MASK] [MASK]'' ) and orthodox'' wide ( also correct but again [MASK] it is only 6'[MASK] wide on [MASK] ) the rubber feet are great for [MASK] - skid purposes, but that's the only positive thing [MASK] have 

In [50]:

downsampled_dataset = lm_datasets

In [51]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['dev'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [52]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/6 [00:00<?, ?it/s]

>>> Perplexity: 21.19


In [53]:
trainer.train()

  0%|          | 0/880 [00:00<?, ?it/s]

{'loss': 3.1944, 'grad_norm': 2.3381853103637695, 'learning_rate': 8.599999999999999e-06, 'epoch': 0.98}


  0%|          | 0/6 [00:00<?, ?it/s]

{'eval_loss': 3.0137174129486084, 'eval_runtime': 0.5211, 'eval_samples_per_second': 308.933, 'eval_steps_per_second': 11.513, 'epoch': 1.0}
{'loss': 2.9801, 'grad_norm': 2.1476967334747314, 'learning_rate': 1.7199999999999998e-05, 'epoch': 1.95}


  0%|          | 0/6 [00:00<?, ?it/s]

...

{'eval_loss': 2.1772449016571045, 'eval_runtime': 0.508, 'eval_samples_per_second': 316.91, 'eval_steps_per_second': 11.81, 'epoch': 16.0}
{'loss': 2.1973, 'grad_norm': 1.9152376651763916, 'learning_rate': 3.921052631578947e-05, 'epoch': 16.61}


  0%|          | 0/6 [00:00<?, ?it/s]

Overwriting existing adapter 'mlm_inv_baby'.


{'eval_loss': 2.2778496742248535, 'eval_runtime': 0.5184, 'eval_samples_per_second': 310.6, 'eval_steps_per_second': 11.575, 'epoch': 17.0}
{'train_runtime': 169.8558, 'train_samples_per_second': 165.434, 'train_steps_per_second': 5.181, 'train_loss': 2.479313141521923, 'epoch': 17.0}


TrainOutput(global_step=748, training_loss=2.479313141521923, metrics={'train_runtime': 169.8558, 'train_samples_per_second': 165.434, 'train_steps_per_second': 5.181, 'train_loss': 2.479313141521923, 'epoch': 17.0})

In [54]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/6 [00:00<?, ?it/s]

>>> Perplexity: 9.21


In [55]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


#### This is a report of mlm for goverment domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 14.10

The results after:
>>> Perplexity: 4.66